In [ ]:
import imageio
import pickle
from PIL import Image
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from tqdm import tqdm
from utils import *
import torch.nn.functional as F

In [ ]:
#Get classes
CLASS_URL = 'https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl'
class_dict = pickle.load(urllib.request.urlopen(CLASS_URL))

#Retrieve image and assign class
#IMG_URL = "https://bloximages.chicago2.vip.townnews.com/chippewa.com/content/tncms/assets/v3/editorial/c/ca/cca0d87d-8d7c-520f-b012-bc065d18b3dd/553cbf5aa0ffb.image.jpg?crop=200%2C200%2C0%2C8&resize=1200%2C1200&order=crop%2Cresize"
#IMG_URL = "https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSa3p29cIlgj0yQwMuLZzh8PKkgBUzbPLWrU-7K79DjDL498JsA&usqp=CAU"
#IMG_URL = "https://thumbs-prod.si-cdn.com/D9GA8QHJQ70nFNQB-AFZ41S-Bs0=/fit-in/1600x0/https://public-media.si-cdn.com/filer/ea/21/ea2159df-9eec-4b05-a3de-356c57e23227/another_airplane_4676723312.jpg"
IMG_URL = "https://5.imimg.com/data5/OX/QV/SF/SELLER-23061093/national-35-model-air-rifle-500x500.jpg"
#classidx = 919
classidx = 504
response = requests.get(IMG_URL)
img = Image.open('images/original/test.jpg')
#img = Image.new('RGB', (3024, 3024), (255, 255, 255))
print(img.size)
imshow(img)

In [ ]:
#Model parameters
model_img_size = 224
model_transform = transforms.Compose([transforms.Resize((model_img_size,model_img_size)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])

pil_to_tensor = transforms.ToTensor()
tensor_to_pil = transforms.ToPILImage()
model_resize = transforms.Resize((224,224))
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)
forward_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

print(type(img))
img_input = model_transform(img)
#img_input = img_input.unsqueeze(0)
#img_input = Variable(img_input)
img_input.shape

In [ ]:
#Model definition
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrained_model = models.resnet101(pretrained=True)
pretrained_model.to(device)
pretrained_model.eval()

#Pass input through model
img_input = img_input.to(device)
pred = pretrained_model(img_input.unsqueeze(0))

#Use softmax to get predicted probability and view it
lay2 = torch.nn.Softmax(dim=1)
prob = lay2(pred)
maxOcls = prob.max(1)
print("Class is {} ({}) with confidence {}%".format(maxOcls.indices.item(),class_dict[maxOcls.indices.item()],maxOcls.values.item()*100))

In [ ]:
def get_object_mask(input_image):
    #print(test[-3][:])
    test = inv_normalize(input_image)
    test = np.uint8(test.numpy()*255).transpose((1,2,0))
    test = 1 - test
    #test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(test, cv2.COLOR_RGB2GRAY)
    th, thgray = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY_INV); 
    #blurred = cv2.GaussianBlur(thgray, (9, 9), 0)
    th, blurred = cv2.threshold(thgray, 70, 255, cv2.THRESH_BINARY_INV)
    edged = cv2.Canny(blurred, 1, 250, L2gradient=True)   
    #imgplot = plt.imshow(blurred, cmap="gray")
    #plt.show()
    return blurred
    #imshow(input_image[0])
    #applying closing function
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    closed = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)    #finding_contours
    (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)   
    #cnts = [] #only for plate class 
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        cv2.drawContours(closed, [approx], -1, (0, 255, 0), 2)    
    th, im_th = cv2.threshold(closed, 254, 255, cv2.THRESH_BINARY_INV);    
    h, w = im_th.shape[:2]
    im_floodfill = im_th.copy()
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0,0), 0);    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)    # Combine the two images to get the foreground.
    #imshow(im_th)
    #plt.show()
    #imshow(im_floodfill_inv)
    #plt.show()
    im_out = im_th ^ im_floodfill_inv
    im_out = cv2.GaussianBlur(im_out, (3, 3), 0)
    return im_out
print(type(img_input))
imshow_tensor(img_input.cpu(),inv_normalize)
#imm = Image.fromarray(get_object_mask(img_input.cpu()))
#imm.save("class919_mask.png")
imgplot = plt.imshow(get_object_mask(img_input.cpu()))
plt.show()

In [ ]:
def imcorr(repla):
    newrep = torch.zeros_like(repla, dtype=torch.float,device=device)
    rp = torch.tensor([-1.25006,  2.40351, -0.15585,  0.00132],device=device)
    gp = torch.tensor([-1.09171,  2.20569, -0.11374, -0.00064],device=device)
    bp = torch.tensor([-1.04359,  1.84006,  0.21149, -0.00933],device=device)
    newrep[0] = rp[0] * torch.pow(repla[0], 3) + rp[1] * torch.pow(repla[0], 2) + rp[2] * repla[0] + rp[3]
    newrep[1] = gp[0] * torch.pow(repla[1], 3) + gp[1] * torch.pow(repla[1], 2) + gp[2] * repla[1] + gp[3]
    newrep[2] = bp[0] * torch.pow(repla[2], 3) + bp[1] * torch.pow(repla[2], 2) + bp[2] * repla[2] + bp[3]
    return newrep

In [ ]:
mask = torch.tensor(get_object_mask(img_input.cpu()), dtype=torch.float, device=device)
mask = mask / torch.max(mask)

In [ ]:
def applyMask(new_w, batch_limits, mask, c_limits, sig_height, conv_size, precision_depth=2):
    
    lay = torch.nn.Conv1d(1,1,conv_size)

    #Manually setting the weights and bias so the  shutter acts as a box filter
    lay.weight.data = torch.full([1,1,conv_size,1], 1/conv_size, requires_grad=True, dtype=torch.float, device=device)
    lay.bias.data = torch.zeros(1, requires_grad=True, dtype=torch.float, device=device)
    
    sz = new_w.shape[1]
    batch = batch_limits[1]-batch_limits[0]
    #stack the signal to fit the input size
    new_w = stack(new_w,sig_height)             
    
    # EOT sampling for ambient light and shift
    c = torch.rand([batch,1,1,1], device=device) * (c_limits[1] - c_limits[0]) + c_limits[0]
    shift = torch.tensor(range(batch_limits[0],batch_limits[1]), dtype=torch.int)
    #shift = torch.from_numpy(np.array(range(0,batch,1)))
    
    #Shift the signal
    new_w = shift_operation(new_w.unsqueeze(0).repeat(batch,1,1,1).view(-1, sig_height, 1), shift).view(batch,3,sig_height,1)
    
    #Fit w into the range [0,1]. new_w is the same as ft
    #new_w = .5 * (torch.tanh(ootn) + 1)
    
    #precision limit
    new_w = diff_round(new_w, precision_depth)
    
    #Convolution of ft and the shutter
    #gy = lay(new_w.unsqueeze(0).view([3,1,228,batch])).view([batch,3,224,1])
    gy = lay(new_w.transpose(0,3).transpose(0,1)).transpose(0,1).transpose(0,3)

    #Mask the signal to only affect the object
    if mask:
        gy_mask = torch.mul(gy,torch.transpose(mask,1,0))
    else:
        gy_mask = gy

    return (c + (1-c)*gy_mask), c

In [ ]:
img_dir = "./images/original/mug/"
#image_list = [Image.open(img_dir+x) for x in os.listdir(img_dir)]
image_list = [Image.open('./images/original/mugfinal.jpg')]
train_image_size = 252
image_tensors = torch.cat([
                pil_to_tensor(
                    x.resize((train_image_size,train_image_size)))
                     .unsqueeze(0) for x in image_list])
image_tensors = image_tensors.to(device)

In [ ]:
def get_image_batch(batch_size, index=None):
    selected_images = torch.randperm(len(image_list))[:batch_size]
    if index!=None:
        return image_tensors[selected_images]
    else:
        if len(index)<batch_size:
            index += random.sample(range(0, len(image_list)), len(image_list)-len(index))
        return image_tensors[index]

In [ ]:
account_resize = True

In [ ]:
#For resize post convolution
if account_resize:
    repeat_size = int(252/252)
    img_n = img.resize((252,252))
    model_img_size = img_n.size[0]
    img_t = pil_to_tensor(img_n)
    img_t = img_t.to(device)
    def resize2d(img, size):
        return (F.adaptive_avg_pool2d(Variable(img), size)).data
    def upsample2d(img, size=224):
        upsample = nn.Upsample(size=size, mode='bilinear', align_corners=False)
        return upsample(torch.unsqueeze(img, 0))[0]
else:
    model_img_size = 224
    img_n = img

In [ ]:
# typical exposure is in form 1/n s. Available: 15, 20, 25 30, 40, 50, 60, 80, 100, 125, 160, 200, 250
exposure = 125 
exp_micros = 1000000/exposure          # get exposure in microseconds
img_ratio = 3024 / model_img_size      # every row in model is img_ratio rows in original image
model_tr = 10 * img_ratio              # multiply real tr (10 micros) by img_ratio to find model tr
conv_size = exp_micros / model_tr      # divide exposure time by tr to find convolution size
conv_size = int(conv_size)             # Need closest integer approximation. Won't cause a significant difference
conv_size

In [ ]:
def getPrediction(img, classid):
    img_input = model_transform(img)
    img_input = img_input.to(device)
    pred = pretrained_model(img_input.unsqueeze(0))

    #Use softmax to get predicted probability and view it
    lay2 = torch.nn.Softmax(dim=1)
    prob = lay2(pred)
    maxOcls = prob.max(1)
    return class_dict[maxOcls.indices.item()], round(maxOcls.values.item()*100, 2)

In [ ]:
model_img_size + conv_size - 1

In [ ]:
image_list = [0]

In [ ]:
def getAdvPrediction(img, classid):
    img_input = model_transform(img)
    img_input = img_input.to(device)
    batch_size = 4
    w = torch.load('saved_inputs/imagenet_504_targeted585_color_resize252.pt', map_location=torch.device('cpu'))
    #w = torch.ones(w.size())
    w = w.to(device)
    w = w[0]
    # For resize post convolution
    if account_resize:
        w = torch.repeat_interleave(w, repeats=repeat_size, dim=1)
    out_prediction = []
    masked_imgs = []
    out_imgs = []
    out_prob = np.array([[0]*1000])
    for j in tqdm(range(0,w.shape[1],batch_size)):
        batch_limits = [j, min(w.shape[1],j+batch_size)]
        sig_height = model_img_size + conv_size - 1
        gy, c = applyMask(w, batch_limits, None, [0,0], sig_height, conv_size, 0)
        
        gy = torch.pow(gy, 1/2.2)
        
        #color correction
        gy = torch.cat([imcorr(i).unsqueeze(0) for i in gy])
        for i in range(0,len(image_list),batch_size):
            if account_resize:
                #For resize post convolution
                inp = gy*img_t
                masked_imgs+=[x.detach().cpu() for x in inp]
                #Gaussian Noise
                #inp = inp + torch.randn(inp.size(),device=device)*random.random()*0.05
                #inp = torch.clamp(inp,min=0,max=1)
                
                #For multiple train images
                #inp = gy*get_image_batch(gy.size()[0], 3024)
                #inp = gy*get_image_batch(gy.size()[0], [k for k in range(i,i+batch_size)])
    
                inp = torch.cat([forward_normalize(upsample2d(i,224)).unsqueeze(0) for i in inp])
            else: 
                inp = gy*inv_normalize(img_input)
                inp = torch.cat([forward_normalize(i).unsqueeze(0) for i in inp])
            
            out = pretrained_model(inp)
            prob = lay2(out)
            #imshow_tensor(inp[1].detach().cpu(),inv_normalize)
            tran = transforms.ToPILImage()
            maxcls = prob.max(1)
            #print(out.topk(2).indices[0])
            out_prob = np.concatenate((out_prob,prob.detach().cpu().numpy()))
            for k in range(batch_limits[1]-batch_limits[0]):
                #print("alpha: {}".format(c[i].item()))
                c_value = round(c[k][0][0][0].data.item(),4)
                out_imgs.append(tran(inv_normalize(inp[k].detach().cpu())))
                out_prediction.append((class_dict[maxcls.indices[k].item()], round(maxcls.values[k].item()*100,4),class_dict[classid],round(prob[k][classid].data.item()*100,4),c_value))
        del out
        del inp
        del gy
        del prob
        del maxcls
    return out_prediction, out_imgs, out_prob, masked_imgs

In [ ]:
out_prediction, out_img, out_prob, masked_imgs = getAdvPrediction(img_n, 504)
#while(len([o for o in out_prediction if o[0]!="street sign"]) < 110):
#    out_prediction, out_img = getAdvPrediction(img, 919)
#    print(len([o for o in out_prediction if o[0]!="street sign"]))
#out_prediction

In [ ]:
#out_prediction

In [ ]:
mean_prob = out_prob.mean(axis=0)
[(class_dict[x],x,mean_prob[x]) for x in np.argsort(mean_prob)[::-1]][:10]

In [ ]:
source_cl = 504
target_cl = 585

In [ ]:
print("dodge_success",(sum([1 for o in out_prediction if o[0]!=class_dict[source_cl]])*100)/len(out_prediction))
print("target_success",(sum([1 for o in out_prediction if o[0]==class_dict[target_cl]])*100)/len(out_prediction))
vals = np.array([o[1] for o in out_prediction if o[0]==class_dict[target_cl]])
print("target_confidence", vals.mean())
vals = np.array([o[1] for o in out_prediction if o[0]==class_dict[source_cl]])
print("source_confidence", vals.mean())

In [ ]:
out_img[1]

In [ ]:
test2 = Image.open('images/original/test2.jpg')
test2 = test2.resize((252,252))
test2

In [ ]:
import imagehash
min_val, min_img = 1000000,None
for o in masked_imgs:
    #print(imagehash.average_hash(test2)-imagehash.average_hash(tensor_to_pil(o)))

In [ ]:
sim = tensor_to_pil(masked_imgs[4])
phy = test2

In [ ]:
x_test = np.linspace(0,200,250)
coeff = np.polyfit(sim_vector,phy_vector,3,full=True)
print(coeff)
p = np.poly1d(coeff)

In [ ]:
sim_vector = np.array(sim)[:,:,0].flatten()
phy_vector = np.array(phy)[:,:,0].flatten()

In [ ]:
import matplotlib.pyplot as plt
f, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(18,5))
ax1.scatter(sim_vector, phy_vector, color='r')
ax1.scatter([i for i in range(250)],[i for i in range(250)], color='k')
ax1.scatter(x_test, p(x_test),color='b')
ax2.scatter(np.array(sim)[:,:,1].flatten(), np.array(phy)[:,:,1].flatten(), color='g')
ax2.scatter([i for i in range(250)],[i for i in range(250)], color='k')
ax3.scatter(np.array(sim)[:,:,2].flatten(), np.array(phy)[:,:,2].flatten(), phy_vector, color='b')
ax3.scatter([i for i in range(250)],[i for i in range(250)], color='k')
plt.show()

In [ ]:
from PIL import ImageChops
ImageChops.difference(test2, tensor_to_pil(masked_imgs[1]))

In [ ]:
without_resize[0]

In [ ]:
after_resize[0]

In [ ]:
lab = np.array([out[0] for out in out_prediction])
np.unique(lab,return_counts=True)

In [ ]:
img_ann_tm = annonate(border(out_img[0]), out_prediction[0])
display(img_ann_tm)
img_ann_tm.save("animation_images/919_0.5.png")

In [ ]:
def writeToGif(filenames,fpss):
    with imageio.get_writer('animation_images/movie.gif', mode='I',fps=fpss) as writer:
        for image in filenames:
            #image = imageio.imread(filename)
            writer.append_data(image)

In [ ]:
def border(old_im):
    #old_im = Image.open('c0.7_sz46_tg861.png')
    old_size = old_im.size
    new_size = (230, 260)
    new_im = Image.new("RGB", new_size,"#FFF")   ## luckily, this is already black!
    new_im.paste(old_im, (3,0))

    #display(new_im)
    return new_im
# new_im.save('someimage.jpg')

In [ ]:
def annonate(img, out):
    draw = ImageDraw.Draw(img)
    # font = ImageFont.truetype(<font-file>, <font-size>)
    #font = ImageFont.load_default()
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 10, encoding="unic")
    # draw.text((x, y),"Sample Text",(r,g,b))
    draw.text((0, 224),"Ambient Light = "+str(out[4]),(0,0,0),font=font)
    draw.text((0, 235),out[0][:15]+" = "+str(out[1]),(0,0,0),font=font)
    draw.text((0, 246),out[2]+" = "+str(out[3]),(0,0,0),font=font)
    #display(img)
    return img
#img.save('sample-out.jpg')

In [ ]:
annonated_imgs = []
for i in range(0,len(out_img),5):
    im = out_img[i]
    #if out_prediction[i][0]=="street sign": continue
    img_ann = annonate(border(im), out_prediction[i])
    #filenames.append("animation_images/shift_"+str(i)+".jpg")
    annonated_imgs.append(np.array(img_ann))
    #img.save("animation_images/shift"+str(i)+".jpg")
writeToGif(annonated_imgs, 5)